In [ ]:
# Install all required packages
!pip install streamlit
!pip install langchain chromadb sentence-transformers faiss-cpu
!pip install gtts
!pip install -U bitsandbytes accelerate transformers
!pip install -U langchain-community
!pip install diffusers
!pip install peft

In [ ]:
# Create HF token file
hf_token = input("Enter your Hugging Face token: ")
with open("hftoken.txt", "w") as f:
    f.write(hf_token)
print("Token saved!")

In [ ]:
import gc
# Create the backend module file
backend_code = '''# Another_copy_of_text_generator_Dataset_creation.py
# Directly extracted & organized from your ipynb

import os, re, datetime, torch, base64
from io import BytesIO
from gtts import gTTS
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from huggingface_hub import login
import random
import gc

# Globals
text_model, tokenizer, image_pipe = None, None, None
quiz_model, quiz_tokenizer = None, None
embeddings, text_splitter = None, None
device = "cuda" if torch.cuda.is_available() else "cpu"

# ======================================================
# hf_token access
# ======================================================
# Read token from local file
with open("hftoken.txt", "r") as f:
    hf_token = f.read().strip()
if not hf_token:
    raise ValueError("HF token file is empty")
else:
    login(hf_token)
    print("✅ Logged into Hugging Face")

# ======================================================
# Initialization
# ======================================================
def initialize_models(hf_model_name="Manoghn/tinyllama-lesson-synthesizer",
                      base_model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                      token_file_path="hftoken.txt"):
    """Initialize lesson + image models using token from local file"""
    global text_model, tokenizer, image_pipe, embeddings, text_splitter

    # Read token from local file
    with open(token_file_path, "r") as f:
        hf_token = f.read().strip()
    if not hf_token:
        raise ValueError("HF token file is empty")

    print("="*60)
    print("STARTING MODEL INITIALIZATION (Memory Optimized)")
    print("="*60)

    # Clear cache before starting
    torch.cuda.empty_cache()
    gc.collect()

    # Tokenizer + model
    tokenizer = AutoTokenizer.from_pretrained(hf_model_name, use_auth_token=hf_token)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        use_auth_token=hf_token
    )

    text_model = PeftModel.from_pretrained(base_model, hf_model_name, torch_dtype=torch.float32, use_auth_token=hf_token).to(device)
    text_model.eval()
    print("✅ Lesson generation model loaded!")

    # Initialize quiz model
    print("\\n🎯 Loading quiz generation model...")
    initialize_quiz_model()

    # Stable Diffusion for image generation
    image_pipe = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        torch_dtype=torch.float16,
        safety_checker=None,
        requires_safety_checker=False,
        variant="fp16"
    )
    image_pipe.scheduler = DPMSolverMultistepScheduler.from_config(image_pipe.scheduler.config)
    image_pipe.enable_model_cpu_offload()  # This replaces .to(device)
    image_pipe.enable_attention_slicing()
    image_pipe.enable_vae_slicing()
    image_pipe.enable_vae_tiling()  # Additional memory optimization

    print("✅ Image model loaded with CPU offloading")

    # Embeddings + text splitter
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    print("✅ Models initialized")

def initialize_quiz_model():
    """Initialize specialized quiz generation model"""
    global quiz_model, quiz_tokenizer

    bnb_config = BitsAndBytesConfig(load_in_8bit=True, bnb_8bit_compute_dtype=torch.float16)
    models_to_try = [
        "mistralai/Mistral-7B-Instruct-v0.1",
        "meta-llama/Llama-2-7b-chat-hf",
        "HuggingFaceH4/zephyr-7b-beta",
        "google/flan-t5-xl",
        "google/flan-t5-large"
    ]

    for model_id in models_to_try:
        try:
            if "flan-t5" in model_id:
                from transformers import T5ForConditionalGeneration, T5Tokenizer
                quiz_tokenizer = T5Tokenizer.from_pretrained(model_id)
                quiz_model = T5ForConditionalGeneration.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16,
                    device_map="auto"
                )
            else:
                quiz_tokenizer = AutoTokenizer.from_pretrained(model_id)
                quiz_model = AutoModelForCausalLM.from_pretrained(
                    model_id, quantization_config=bnb_config, device_map="auto"
                )
            print(f"✅ Quiz model loaded: {model_id}")
            return
        except Exception as e:
            print(f"⚠️ Could not load {model_id}: {e}")

    quiz_model = None
    quiz_tokenizer = None

# ======================================================
# Lesson generation
# ======================================================
def generate_lesson(topic, max_length=800):
    """Generate educational lesson text"""
    global text_model, tokenizer
    if text_model is None:
        initialize_models()

    prompt = f"<human>: Create a comprehensive educational lesson about the topic: {topic}\\n<assistant>:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = text_model.generate(
            inputs.input_ids,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("<assistant>:")[-1].strip()

# ======================================================
# Image generation
# ======================================================
def get_image_suggestions(lesson_text, topic, max_images=3):
    """Analyze lesson text & suggest image prompts"""
    prompts = [
        f"An educational diagram illustrating {topic}, high quality, professional",
        f"A visual representation of {topic} concept, clear and detailed",
        f"Simple educational illustration explaining {topic}, clean design"
    ]
    return prompts[:max_images]

def generate_images(suggestions):
    """Generate images from text prompts"""
    global image_pipe
    if image_pipe is None:
        initialize_models()

    images_b64 = []
    for suggestion in suggestions:
        try:
            image = image_pipe(
                suggestion,
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]
            buf = BytesIO()
            image.save(buf, format="PNG")
            buf.seek(0)
            images_b64.append("data:image/png;base64," + base64.b64encode(buf.getvalue()).decode())
        except Exception as e:
            print(f"Error generating image: {e}")
            # Create a placeholder image
            from PIL import Image, ImageDraw
            img = Image.new('RGB', (512, 512), color='white')
            draw = ImageDraw.Draw(img)
            draw.text((100, 256), "Image Generation Failed", fill='black')
            buf = BytesIO()
            img.save(buf, format="PNG")
            buf.seek(0)
            images_b64.append("data:image/png;base64," + base64.b64encode(buf.getvalue()).decode())
    return images_b64

# ======================================================
# Enhanced Quiz generation from notebook
# ======================================================
def generate_quiz_questions(lesson_text, topic, num_questions=15):
    """Generate multiple types of quiz questions using specialized model"""
    print(f"\\n❓ Generating {num_questions} quiz questions with multiple formats...")

    # Initialize quiz model if not already loaded
    if quiz_model is None:
        initialize_quiz_model()

    # Determine question type distribution
    question_distribution = {
        "multiple_choice": int(num_questions * 0.3),  # 30%
        "true_false": int(num_questions * 0.25),      # 25%
        "fill_blank": int(num_questions * 0.25),      # 25%
        "short_answer": int(num_questions * 0.2)      # 20%
    }

    # Adjust for rounding
    total = sum(question_distribution.values())
    if total < num_questions:
        question_distribution["multiple_choice"] += num_questions - total

    print(f"📊 Question distribution: {question_distribution}")

    all_questions = []

    # Generate each type of question
    for q_type, count in question_distribution.items():
        if count > 0:
            print(f"\\n🎲 Generating {count} {q_type} questions...")

            if quiz_model is not None:
                questions = generate_questions_by_type(lesson_text, topic, q_type, count)
            else:
                questions = generate_fallback_by_type(lesson_text, topic, q_type, count)

            all_questions.extend(questions)
            print(f"✅ Generated {len(questions)} {q_type} questions")

    # Shuffle for variety
    random.shuffle(all_questions)

    return all_questions[:num_questions]

def generate_questions_by_type(lesson_text, topic, question_type, count):
    """Generate specific question types using the quiz model"""

    # Create type-specific prompts
    if question_type == "multiple_choice":
        prompt = f"""Generate {count} multiple choice questions about {topic}.

Based on this lesson:
{lesson_text[:1000]}

Format each question exactly like this:
Question 1: [question text]
A) [option]
B) [option]
C) [option]
D) [option]
Correct: [A/B/C/D]

Question 2: [question text]
A) [option]
B) [option]
C) [option]
D) [option]
Correct: [A/B/C/D]"""

    elif question_type == "true_false":
        prompt = f"""Generate {count} true/false questions about {topic}.

Based on this lesson:
{lesson_text[:1000]}

Format each question exactly like this:
Question 1: [statement]
Answer: True/False

Question 2: [statement]
Answer: True/False"""

    elif question_type == "fill_blank":
        prompt = f"""Generate {count} fill-in-the-blank questions about {topic}.

Based on this lesson:
{lesson_text[:1000]}

Format each question exactly like this:
Question 1: [sentence with _____ for the blank]
Answer: [missing word/phrase]

Question 2: [sentence with _____ for the blank]
Answer: [missing word/phrase]"""

    else:  # short_answer
        prompt = f"""Generate {count} short answer questions about {topic}.

Based on this lesson:
{lesson_text[:1000]}

Format each question exactly like this:
Question 1: [question requiring 1-2 sentence answer]
Answer: [brief answer]

Question 2: [question requiring 1-2 sentence answer]
Answer: [brief answer]"""

    try:
        # Format prompt based on model
        if quiz_model is None:
            return generate_fallback_by_type(lesson_text, topic, question_type, count)

        model_name = quiz_model.config._name_or_path

        if "flan-t5" in model_name:
            formatted_prompt = prompt
        elif "mistral" in model_name.lower():
            formatted_prompt = f"<s>[INST] {prompt} [/INST]"
        elif "llama" in model_name.lower():
            formatted_prompt = f"<s>[INST] <<SYS>>\\nYou are a helpful teacher creating quiz questions.\\n<</SYS>>\\n\\n{prompt} [/INST]"
        else:
            formatted_prompt = prompt

        inputs = quiz_tokenizer(formatted_prompt, return_tensors="pt", max_length=1500, truncation=True)

        with torch.no_grad():
            outputs = quiz_model.generate(
                inputs.input_ids.to(device),
                max_length=2500,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=quiz_tokenizer.eos_token_id
            )

        response = quiz_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract generated content
        if "[/INST]" in response:
            response = response.split("[/INST]", 1)[1].strip()

        # Parse based on question type
        return parse_questions_by_type(response, topic, question_type)

    except Exception as e:
        print(f"⌛ Model generation failed: {e}")
        return generate_fallback_by_type(lesson_text, topic, question_type, count)

def parse_questions_by_type(response, topic, question_type):
    """Parse different question types from model response"""
    questions = []

    if question_type == "multiple_choice":
        # Parse MCQ format
        pattern = r'Question\\s*\\d*:?\\s*(.+?)\\n\\s*A\\)\\s*(.+?)\\n\\s*B\\)\\s*(.+?)\\n\\s*C\\)\\s*(.+?)\\n\\s*D\\)\\s*(.+?)\\n\\s*Correct:\\s*([A-D])'
        matches = re.findall(pattern, response, re.MULTILINE | re.DOTALL)

        for match in matches:
            q_text, opt_a, opt_b, opt_c, opt_d, correct = match
            questions.append({
                "type": "multiple_choice",
                "question": q_text.strip(),
                "options": {
                    "A": opt_a.strip(),
                    "B": opt_b.strip(),
                    "C": opt_c.strip(),
                    "D": opt_d.strip()
                },
                "correct": correct.upper(),
                "answer": f"The correct answer is {correct.upper()}",
                "concept": f"{topic} MCQ"
            })

    elif question_type == "true_false":
        # Parse T/F format
        pattern = r'Question\\s*\\d*:?\\s*(.+?)\\n\\s*Answer:\\s*(True|False)'
        matches = re.findall(pattern, response, re.IGNORECASE)

        for statement, answer in matches:
            questions.append({
                "type": "true_false",
                "question": f"True or False: {statement.strip()}",
                "answer": answer.capitalize(),
                "concept": f"{topic} T/F"
            })

    elif question_type == "fill_blank":
        # Parse fill-in-the-blank format
        pattern = r'Question\\s*\\d*:?\\s*(.+?_+.+?)\\n\\s*Answer:\\s*(.+?)(?:\\n|$)'
        matches = re.findall(pattern, response, re.MULTILINE)

        for blank_q, answer in matches:
            questions.append({
                "type": "fill_blank",
                "question": blank_q.strip(),
                "answer": answer.strip(),
                "concept": f"{topic} Fill"
            })

    else:  # short_answer
        # Parse short answer format
        pattern = r'Question\\s*\\d*:?\\s*(.+?)\\n\\s*Answer:\\s*(.+?)(?=Question|\\Z)'
        matches = re.findall(pattern, response, re.DOTALL)

        for q_text, answer in matches:
            questions.append({
                "type": "short_answer",
                "question": q_text.strip(),
                "answer": answer.strip()[:200],  # Limit answer length
                "concept": f"{topic} Short"
            })

    return questions

def generate_fallback_by_type(lesson_text, topic, question_type, count):
    """Generate specific question types without specialized model"""
    questions = []
    sentences = [s.strip() for s in lesson_text.split('.') if len(s.strip()) > 30]

    if question_type == "multiple_choice":
        # Generate MCQs from lesson content
        for i in range(min(count, len(sentences))):
            sentence = sentences[i % len(sentences)]

            # Extract a fact that can be questioned
            if " is " in sentence:
                parts = sentence.split(" is ", 1)
                subject = parts[0].strip()
                predicate = parts[1].strip()

                questions.append({
                    "type": "multiple_choice",
                    "question": f"What is {subject}?",
                    "options": {
                        "A": predicate,
                        "B": f"Not related to {topic}",
                        "C": f"A type of {topic} equipment",
                        "D": f"The opposite of {subject}"
                    },
                    "correct": "A",
                    "answer": "The correct answer is A",
                    "concept": f"{topic} MCQ"
                })

    elif question_type == "true_false":
        # Generate T/F from sentences
        for i in range(min(count, len(sentences))):
            sentence = sentences[i % len(sentences)]

            if i % 2 == 0:
                # True statement
                questions.append({
                    "type": "true_false",
                    "question": f"True or False: {sentence}",
                    "answer": "True",
                    "concept": f"{topic} T/F"
                })
            else:
                # Create false statement
                false_sentence = sentence.replace(" is ", " is not ")
                if false_sentence == sentence:
                    false_sentence = sentence.replace(" are ", " are not ")

                questions.append({
                    "type": "true_false",
                    "question": f"True or False: {false_sentence}",
                    "answer": "False",
                    "concept": f"{topic} T/F"
                })

    elif question_type == "fill_blank":
        # Generate fill-in-the-blank
        key_terms = re.findall(r'\\*\\*([^*]+)\\*\\*', lesson_text)

        for i in range(min(count, len(sentences))):
            sentence = sentences[i % len(sentences)]
            words = sentence.split()

            if len(words) > 5:
                # Replace a key word with blank
                if i < len(key_terms) and key_terms[i] in sentence:
                    blank_sentence = sentence.replace(key_terms[i], "_____")
                    answer = key_terms[i]
                else:
                    # Replace middle word
                    blank_pos = len(words) // 2
                    answer = words[blank_pos]
                    words[blank_pos] = "_____"
                    blank_sentence = " ".join(words)

                questions.append({
                    "type": "fill_blank",
                    "question": blank_sentence,
                    "answer": answer,
                    "concept": f"{topic} Fill"
                })

    else:  # short_answer
        # Generate short answer questions
        question_templates = [
            f"What is the main purpose of {topic}?",
            f"How does {topic} work?",
            f"Why is {topic} important?",
            f"Describe the process of {topic}.",
            f"What are the key components of {topic}?"
        ]

        for i in range(min(count, len(question_templates))):
            questions.append({
                "type": "short_answer",
                "question": question_templates[i],
                "answer": sentences[i % len(sentences)][:150] if sentences else f"{topic} is an important concept.",
                "concept": f"{topic} Short"
            })

    return questions[:count]

# ======================================================
# Audio narration
# ======================================================
def generate_audio_base64(text: str, topic: str = "lesson") -> str:
    """Generate audio narration using gTTS"""
    try:
        clean_text = re.sub(r'[#*\\[\\]]', '', text)
        clean_text = re.sub(r'\\n+', '. ', clean_text)
        tts = gTTS(text=f"Lesson on {topic}. {clean_text[:500]}", lang='en', slow=False)
        audio_bytes = BytesIO()
        tts.write_to_fp(audio_bytes)
        audio_bytes.seek(0)
        return base64.b64encode(audio_bytes.getvalue()).decode()
    except Exception as e:
        print(f"Audio generation failed: {e}")
        return ""

# ======================================================
# Enhanced Study guide creation from notebook
# ======================================================
def create_study_guide(topic, lesson_text, quiz_questions):
    """Create comprehensive study guide with multiple question types"""
    study_guide = f"""# 📚 Study Guide: {topic}

## 📝 Lesson Summary

{lesson_text[:500]}...

---

## ❓ Practice Questions ({len(quiz_questions)} questions)

"""

    # Group questions by type
    questions_by_type = {}
    for q in quiz_questions:
        q_type = q.get('type', 'simple_qa')
        if q_type not in questions_by_type:
            questions_by_type[q_type] = []
        questions_by_type[q_type].append(q)

    type_names = {
        "multiple_choice": "🔤 Multiple Choice Questions",
        "true_false": "✅ True or False",
        "fill_blank": "📝 Fill in the Blanks",
        "short_answer": "💭 Short Answer Questions",
        "simple_qa": "❓ Questions & Answers"
    }

    question_num = 1

    # Display questions by type
    for q_type, type_questions in questions_by_type.items():
        if type_questions:
            study_guide += f"\\n### {type_names.get(q_type, q_type)} ({len(type_questions)} questions)\\n\\n"

            for q in type_questions:
                # Clean up text
                question_text = q.get('question', '').strip()
                if len(question_text) > 300:
                    question_text = question_text[:300] + "..."

                if q_type == "multiple_choice":
                    study_guide += f"""**{question_num}.** {question_text}

A) {q['options'].get('A', 'Option A')}
B) {q['options'].get('B', 'Option B')}
C) {q['options'].get('C', 'Option C')}
D) {q['options'].get('D', 'Option D')}

<details>
<summary>Show Answer</summary>

✅ **{q.get('correct', 'A')}** - {q['options'].get(q.get('correct', 'A'), 'Correct answer')}

</details>

"""
                elif q_type == "true_false":
                    study_guide += f"""**{question_num}.** {question_text}

<details>
<summary>Show Answer</summary>

✅ **{q.get('answer', 'True')}**

</details>

"""
                elif q_type == "fill_blank":
                    study_guide += f"""**{question_num}.** {question_text}

<details>
<summary>Show Answer</summary>

✅ **{q.get('answer', 'Answer')}**

</details>

"""
                else:  # short_answer or simple_qa
                    answer_text = q.get('answer', 'Answer not provided').strip()
                    if len(answer_text) > 300:
                        answer_text = answer_text[:300] + "..."

                    study_guide += f"""**{question_num}.** {question_text}

<details>
<summary>Show Answer</summary>

{answer_text}

</details>

"""
                question_num += 1

            study_guide += "---\\n"

    # Add statistics section
    study_guide += f"""## 📊 Question Statistics

- **Total Questions**: {len(quiz_questions)}
- **Question Types**: {len(questions_by_type)}
"""

    # Calculate percentages for each type
    for q_type, type_questions in questions_by_type.items():
        percentage = (len(type_questions) / len(quiz_questions)) * 100
        study_guide += f"- **{type_names.get(q_type, q_type)}**: {len(type_questions)} ({percentage:.0f}%)\\n"

    # Add study tips
    study_guide += """
---

## 💡 Study Tips

### By Question Type:

**🔤 Multiple Choice Tips:**
- Read all options before selecting
- Eliminate obviously wrong answers first
- Look for keywords in the question

**✅ True/False Tips:**
- Watch for absolute words (always, never, all, none)
- Look for qualifiers that might change meaning

**📝 Fill in the Blanks Tips:**
- Read the entire sentence first
- Consider the context around the blank
- Check if your answer makes grammatical sense

**💭 Short Answer Tips:**
- Answer in complete sentences
- Include key terms from the lesson
- Be concise but thorough

---

*Generated on: """ + datetime.datetime.now().strftime("%B %d, %Y at %I:%M %p") + "*"

    return study_guide

# ======================================================
# Main Study guide generation wrapper
# ======================================================
def generate_study_guide(topic: str, num_images=3, num_questions=15):
    """High-level wrapper to generate full study package"""
    # 1. Lesson
    lesson = generate_lesson(topic)

    # 2. Image prompts + images
    suggestions = get_image_suggestions(lesson, topic, max_images=num_images)
    images = generate_images(suggestions)

    # 3. Audio narration
    audio = generate_audio_base64(lesson, topic)

    # 4. Enhanced quiz questions with multiple types
    quiz = generate_quiz_questions(lesson, topic, num_questions=num_questions)

    # 5. Create comprehensive study guide
    study_guide_text = create_study_guide(topic, lesson, quiz)

    return {
        "topic": topic,
        "lesson": lesson,
        "images": images,
        "audio": audio,
        "quiz": quiz,
        "study": study_guide_text,
    }
'''

with open("Another_copy_of_text_generator_Dataset_creation.py", "w", encoding='utf-8') as f:
    f.write(backend_code)
print("Backend module created!")

In [ ]:
# Create study_generator.py
with open("study_generator.py", "w") as f:
    f.write("""from Another_copy_of_text_generator_Dataset_creation import generate_study_guide

__all__ = ["generate_study_guide"]
""")
print("Study generator module created!")

In [ ]:
# Create app.py
app_code = '''# app.py
# app.py
import streamlit as st
import base64
import study_generator as sg

# --- Page setup ---
st.set_page_config(page_title="Study Guide Chat", layout="centered")

# --- JS to detect theme ---
st.markdown("""
<script>
const theme = window.parent.document.querySelector('body').classList.contains('dark') ? 'dark' : 'light';
window.parent.postMessage({func: 'setTheme', theme: theme}, '*');
</script>
""", unsafe_allow_html=True)

if "theme" not in st.session_state:
    st.session_state.theme = "light"  # default theme

# --- Import 21st.dev icons ---
st.markdown("""
<script type="module" src="https://unpkg.com/@21st-dev/icons@latest/dist/icons.esm.js"></script>
""", unsafe_allow_html=True)

# --- Dynamic colors ---
theme = st.session_state.get("theme", "light")
if theme == "dark":
    card_bg = "#1e1e1e"
    text_color = "#f5f5f5"
    border_color = "#6b4c3b"
    hover_border = "#d6c1b1"
else:
    card_bg = "#ffffff"
    text_color = "#111111"
    border_color = "#d6c1b1"
    hover_border = "#6b4c3b"

# --- Custom CSS ---
st.markdown(f"""
<style>
body {{
    font-family: Inter, sans-serif;
    background: {card_bg};
    color: {text_color};
}}
.block-container {{
    padding-top: 2rem;
    padding-bottom: 2rem;
}}
.card {{
    background: {card_bg} !important;
    color: {text_color};
    border-radius: 14px;
    padding: 20px;
    margin: 14px 0;
    border: 1px solid {border_color};
    box-shadow: 0 2px 6px rgba(0,0,0,0.05);
    transition: all 0.3s ease-in-out;
}}
.card:hover {{
    box-shadow: 0 8px 20px rgba(0,0,0,0.12);
    transform: translateY(-3px);
    border-color: {hover_border};
}}
.section-title {{
    margin-top: 18px;
    font-weight: 600;
    font-size: 15px;
    color: {hover_border};
}}
.stButton>button {{
    background: transparent;
    border: none;
    font-weight: 600;
    font-size: 16px;
    text-align: left;
    width: 100%;
    color: {text_color};
    cursor: pointer;
    padding: 0;
}}
.stButton>button:hover {{
    color: {hover_border};
}}
audio {{
    margin-top: 8px;
}}
</style>
""", unsafe_allow_html=True)

# --- Title ---
st.title("Study Guide Chat")

# --- Session state ---
if "messages" not in st.session_state:
    st.session_state.messages = []

# --- Chat input ---
topic = st.chat_input("Enter a topic to study")

if topic:
    with st.spinner("Generating study guide..."):
        guide = sg.generate_study_guide(topic)
        st.session_state.messages.append({
            "topic": guide["topic"],
            "lesson": guide["lesson"],
            "images": guide["images"],
            "audio": guide["audio"],
            "quiz": guide["quiz"],
            "study": guide["study"],
            "open": False
        })

# --- Display tiles ---
for idx, msg in enumerate(st.session_state.messages):
    with st.container():
        st.markdown('<div class="card">', unsafe_allow_html=True)

        # Toggle
        if st.button(f"{msg['topic']} ▸" if not msg["open"] else f"{msg['topic']} ▾", key=f"toggle_{idx}"):
            msg["open"] = not msg["open"]

        # Expanded view
        if msg["open"]:
            st.markdown('<div class="section-title"><tw-icon name="book"></tw-icon> Lesson</div>', unsafe_allow_html=True)
            st.write(msg["lesson"])

            st.markdown('<div class="section-title"><tw-icon name="image"></tw-icon> Images</div>', unsafe_allow_html=True)
            cols = st.columns(max(1, len(msg["images"])))
            for i, img in enumerate(msg["images"]):
                with cols[i]:
                    st.image(img, use_container_width=True)

            if msg["audio"]:
                st.markdown('<div class="section-title"><tw-icon name="volume-high"></tw-icon> Audio</div>', unsafe_allow_html=True)
                st.audio(base64.b64decode(msg["audio"]), format="audio/mp3")

            st.markdown('<div class="section-title"><tw-icon name="clipboard-list"></tw-icon> Quiz</div>', unsafe_allow_html=True)
            for q in msg["quiz"]:
                if isinstance(q, dict):
                    # Handle both old format ('q') and new format ('question')
                    question_text = q.get('question', q.get('q', 'Question'))
                    answer_text = q.get('answer', q.get('a', 'Answer'))

                    st.write(f"• {question_text}")
                    # Optionally show answers in a collapsible
                    with st.expander("Show answer"):
                        st.write(f"Answer: {answer_text}")
                else:
                    st.write(f"• {q}")

            st.markdown('<div class="section-title"><tw-icon name="document-text"></tw-icon> Study Guide</div>', unsafe_allow_html=True)
            st.write(msg["study"])

        st.markdown('</div>', unsafe_allow_html=True)

'''

with open("app.py", "w", encoding='utf-8') as f:
    f.write(app_code)
print("Streamlit app created!")

In [ ]:
# Install and setup ngrok
!pip install pyngrok
from pyngrok import ngrok

# Get ngrok authtoken (you'll need to sign up at ngrok.com for free)
ngrok_token = input("Enter your ngrok authtoken (get it from ngrok.com): ")
ngrok.set_auth_token(ngrok_token)

In [ ]:
# Let's test if your code is even running
print("Testing backend...")

try:
    import Another_copy_of_text_generator_Dataset_creation as backend
    print("✅ Backend loaded!")

    # Initialize models (this might take a minute)
    print("\n⏳ Initializing models (this takes 1-2 minutes)...")
    backend.initialize_models()
    print("✅ Models loaded!")

    # Quick test
    print("\n📝 Generating test lesson...")
    lesson = backend.generate_lesson("test topic", max_length=100)
    print(f"Success! Generated: {lesson[:50]}...")

except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
import subprocess
from threading import Thread
import time
from pyngrok import ngrok

# Kill all existing tunnels
for tunnel in ngrok.get_tunnels():
    print(f"Closing tunnel: {tunnel.public_url}")
    ngrok.disconnect(tunnel.public_url)

# Kill ngrok process completely
ngrok.kill()

print("✅ All tunnels closed")

# Function to run streamlit
def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])

# Start streamlit in background
thread = Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# Wait for streamlit to start
time.sleep(5)

# Create public URL
public_url = ngrok.connect(8501, bind_tls=True)
print(f"\n🚀 Your app is live at: {public_url}")
print("\nNote: The first load might take a few minutes as models are being initialized.")

In [ ]:
# Install localtunnel
!npm install -g localtunnel

# Kill existing streamlit
!pkill -f streamlit

# Start Streamlit in background with logging
!nohup streamlit run app.py --server.port 8501 > streamlit_output.log 2>&1 &

# Wait for startup
import time
time.sleep(5)

# Get your public IP (this is the password for localtunnel)
import requests
public_ip = requests.get('https://api.ipify.org').text
print(f"Your localtunnel password is: {public_ip}")
print("Copy this IP address - you'll need it when the browser asks for a password\n")

# Start localtunnel
!lt --port 8501